### RNN을 이용한 감성분석
영화평으로 부터 평점을 예측하는 모델이 잘 알려져 있습니다.  

1. 데이터 : IMDB movie-review classification problem
    * 긍정, 부정으로만 분리
2. 사용 모델 : RNN, LSTM  

module import 에서 세부적인 함수, 클래스를 직접 호출하면 사용 시 전체 모듈명의 생략이 가능합니다.  

In [1]:
import keras

from keras.models import Sequential
from keras.layers import Embedding, Dense
from keras.layers import SimpleRNN, LSTM
from keras.preprocessing import sequence

from keras.datasets import imdb

/home/bwlee/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


각 단어를 정형화 하기 위해 index 로 표시합니다.  
사용하고자 하는 단어의 수를 정하고, 이 단어 수를 벗어날 만큼 사용 빈도가 낮은 단어는 무시하는 것이 일반적입니다.  
아래의 변수 num_words 가 이 역할을 합니다.  
문장의 길이가 각 instance 마다 다르므로 전체 단어 갯수에 대한 큰 값을 하나 정하고 그 이상 넘어가는 문장의 단어는 무시합니다.  
sequence.pad_sequences 함수가 maxlen 만큼의 단어들을 골라냅니다. 이 때 앞쪽을 무시할 지, 뒷쪽을 무시할 지는 사용자가 입력으로 넣어주게 됩니다.  
Default 로는 앞쪽을 무시합니다. 아래의 결과에서 확인 가능합니다.  
단어수가 그 이하일 경우 빈 공간은 0 혹은 사용자 지정값으로 채워 줍니다.

In [2]:
num_words = 20000
maxlen = 80
batch_size = 32

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)
print(x_train.shape, x_test.shape)
print(x_train[0])
word2ix = imdb.get_word_index()
word2ix = {word: (ix+3) for word, ix in word2ix.items()}
word2ix["<PAD>"] = 0
word2ix["<START>"] = 1
word2ix["<UNK>"] = 2
ix2word = {i : word for word, i in word2ix.items()}
text = ' '.join([ix2word[i] for i in x_train[0]])
print(text)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print(x_train.shape, x_test.shape)
print(x_train[0])

(25000,) (25000,)
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
1646592/1641221 [=============

#### Embedding
index 로 표현된 단어들은 데이터에 서로 간 연관성이 나타나지 않습니다.  
단어들을 적은 차원의 데이터 값들로 표현할 수 있습니다. 이 때 연관된 단어들은 가까운 위치로 mapping 하게 됩니다.  

In [3]:
def rnn_network():
    model = Sequential()
    model.add(Embedding(num_words, 128))
    model.add(SimpleRNN(128))
    model.add(Dense(1, activation='sigmoid'))
    return model

rnn = rnn_network()
rnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 128)               32896     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 2,593,025
Trainable params: 2,593,025
Non-trainable params: 0
_________________________________________________________________


In [4]:
rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
history = rnn.fit(x_train, y_train, batch_size=batch_size, epochs=10)

Epoch 1/10
25000/25000 [==============================] - 13s 503us/step - loss: 0.5951 - acc: 0.6653
Epoch 2/10
25000/25000 [==============================] - 12s 472us/step - loss: 0.4434 - acc: 0.8026
Epoch 3/10
25000/25000 [==============================] - 12s 466us/step - loss: 0.3377 - acc: 0.8608
Epoch 4/10
25000/25000 [==============================] - 12s 467us/step - loss: 0.2415 - acc: 0.9064
Epoch 5/10
25000/25000 [==============================] - 12s 467us/step - loss: 0.1892 - acc: 0.9290
Epoch 6/10
25000/25000 [==============================] - 12s 468us/step - loss: 0.2754 - acc: 0.8859
Epoch 7/10
25000/25000 [==============================] - 12s 468us/step - loss: 0.1435 - acc: 0.9480
Epoch 8/10
25000/25000 [==============================] - 12s 470us/step - loss: 0.0876 - acc: 0.9705
Epoch 9/10
25000/25000 [==============================] - 12s 469us/step - loss: 0.0822 - acc: 0.9717
Epoch 10/10
25000/25000 [==============================] - 12s 470us/step - loss: 

In [6]:
def lstm_network():
    model = Sequential()
    model.add(Embedding(num_words, 128))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

rnn2 = lstm_network()
rnn2.summary()
rnn2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


In [7]:
history2 = rnn2.fit(x_train, y_train, batch_size=batch_size, epochs=10)

Epoch 1/10
25000/25000 [==============================] - 53s 2ms/step - loss: 0.4605 - acc: 0.7861
Epoch 2/10
25000/25000 [==============================] - 53s 2ms/step - loss: 0.3050 - acc: 0.8744
Epoch 3/10
25000/25000 [==============================] - 53s 2ms/step - loss: 0.2205 - acc: 0.9148
Epoch 4/10
25000/25000 [==============================] - 54s 2ms/step - loss: 0.1530 - acc: 0.9433
Epoch 5/10
25000/25000 [==============================] - 54s 2ms/step - loss: 0.1136 - acc: 0.9587
Epoch 6/10
25000/25000 [==============================] - 54s 2ms/step - loss: 0.0839 - acc: 0.9699
Epoch 7/10
25000/25000 [==============================] - 54s 2ms/step - loss: 0.0612 - acc: 0.9796
Epoch 8/10
25000/25000 [==============================] - 55s 2ms/step - loss: 0.0489 - acc: 0.9844
Epoch 9/10
25000/25000 [==============================] - 55s 2ms/step - loss: 0.0389 - acc: 0.9868
Epoch 10/10
25000/25000 [==============================] - 55s 2ms/step - loss: 0.0252 - acc: 0.9921